<a href="https://colab.research.google.com/github/labsurde/study/blob/master/While_if_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![대체 텍스트](https://)##### Copyright 2019 The TensorFlow Authors.


In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

Download and install TensorFlow 2. Import TensorFlow into your program:

Note: Upgrade `pip` to install the TensorFlow 2 package. See the [install guide](https://www.tensorflow.org/install) for details.

In [0]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

Import TensorFlow into your program:

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals, unicode_literals

# tensorflow와 tf.keras를 임포트합니다
import tensorflow as tf
from tensorflow import keras

print(tf.__version__)

loading data

In [0]:
# load test data set
fashion_mnist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

# let's shrink the data because we don't need full training and we just need tflite graph
train_images = train_images[0:1]
train_labels = train_labels[0:1]
test_images = test_images[0:1]
test_labels = test_labels[0:1]

train_images = train_images / 255.0
test_images = test_images / 255.0               

# Generating while.tflite

In [0]:
# layer for "while"
from tensorflow.keras import layers

class WhileLayer(layers.Layer):

  def __init__(self):
    super(WhileLayer, self).__init__()

  @tf.function
  def call(self, matrix):
    for i in tf.range(5):
      matrix = matrix + 1
    return matrix


In [0]:
# make model and train
model = keras.Sequential([
  keras.layers.Flatten(input_shape=(28, 28)),
  WhileLayer(),
  keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_images, train_labels, epochs=1)

In [0]:
# write tflite file
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.experimental_new_converter = True
tflite_model = converter.convert()

open("while.tflite", "wb").write(tflite_model)


In [0]:
tf.keras.utils.plot_model(model, show_shapes=True)

# generating if.tflite

In [0]:
# layer for "if"
class IfLayer(layers.Layer):

  def __init__(self):
    super(IfLayer, self).__init__()
    self.count = 0

  @tf.function
  def call(self, matrix):
    cond = matrix[0,0] > 0
    if cond:
      matrix = matrix + 1
      return matrix
    else: 
      matrix = matrix - 1
      return matrix

In [0]:
# make model and train
model = keras.Sequential([
  keras.layers.Flatten(input_shape=(28, 28)),
  IfLayer(),
  keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_images, train_labels, epochs=1)

In [0]:
# write tflite file
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.experimental_new_converter = True
tflite_model = converter.convert()

open("if.tflite", "wb").write(tflite_model)

In [0]:
tf.keras.utils.plot_model(model, show_shapes=True)

Now, download `if.tflite` and `while.tflite` from left "FILE" explorer